## Fase 1: Extracción de datos de películas

In [1]:
## Importación de librerías

import pandas as pd
import numpy as np
import requests
import mysql.connector
from mysql.connector import errorcode
pd.set_option('display.max_columns', None) 
import warnings
warnings.filterwarnings("ignore")

In [2]:
## Obtención datos: 100 películas

url = "https://beta.adalab.es/resources/apis/pelis/pelis.json"

datos = requests.get(url)
print(datos.status_code)
print(datos.content)

datos_json = datos.json()

200
b'[\n    {\n        "id": 1,\n        "titulo": "The Godfather",\n        "a\xc3\xb1o": 1972,\n        "duracion": 175,\n        "genero": "Crimen",\n        "adultos": false,\n        "subtitulos": [\n            "es",\n            "en"\n        ]\n    },\n    {\n        "id": 2,\n        "titulo": "The Godfather Part II",\n        "a\xc3\xb1o": 1974,\n        "duracion": 202,\n        "genero": "Crimen",\n        "adultos": false,\n        "subtitulos": [\n            "es",\n            "en"\n        ]\n    },\n    {\n        "id": 3,\n        "titulo": "Pulp Fiction",\n        "a\xc3\xb1o": 1994,\n        "duracion": 154,\n        "genero": "Crimen",\n        "adultos": true,\n        "subtitulos": [\n            "es",\n            "en"\n        ]\n    },\n    {\n        "id": 4,\n        "titulo": "Forrest Gump",\n        "a\xc3\xb1o": 1994,\n        "duracion": 142,\n        "genero": "Drama",\n        "adultos": false,\n        "subtitulos": [\n            "es",\n            

In [3]:
print (type(datos_json))
print (len(datos_json)) ## para saber cuántas hay ya que se pide extraer 100 películas

<class 'list'>
100


In [ ]:
df = pd.DataFrame(datos_json)
df

In [5]:
las_100_pelis = datos_json[:100]

In [6]:
df_100 = pd.DataFrame(las_100_pelis)

## Fase 2: Creación de la Base de Datos

In [7]:
cnx = mysql.connector.connect(user='root', password='1234',
                              host='localhost',
                              database ='examen_apis',auth_plugin='mysql_native_password')

mycursor = cnx.cursor()
print("Conexión OK:", cnx.is_connected())

Conexión OK: True


In [13]:
df_pelis = df_100[["titulo", "año", "duracion", "genero", "adultos"]]
df_pelis.head()

,titulo,año,duracion,genero,adultos
0,The Godfather,1972,175,Crimen,False
1,The Godfather Part II,1974,202,Crimen,False
2,Pulp Fiction,1994,154,Crimen,True
3,Forrest Gump,1994,142,Drama,False
4,The Dark Knight,2008,152,Acción,False


In [10]:
mycursor.execute("USE examen_apis")

query = '''CREATE TABLE IF NOT EXISTS peliculas (
    id INT PRIMARY KEY AUTO_INCREMENT,
    titulo VARCHAR(255) NOT NULL,
    año INT,
    duracion INT,
    genero VARCHAR(50),
    adultos BOOLEAN
);
'''

mycursor.execute(query)
cnx.commit()  ## para que se guarden los datos

## Fase 3: Inserción de los Datos en la Base de Datos

In [ ]:
df_to_insert = df_100[["titulo", "año", "duracion", "genero", "adultos"]].copy() # para no usar columnas que no se van a inssertar

df_clean = df_to_insert.replace({np.nan: None})   # Reemplazar NaN con None para que se inserten como NULL en la BD

df_clean["adultos"] = df_clean["adultos"].map({True: 1, False: 0, None: None}) # para convertir el booleano a 1 = Adultos /0 o no existe

datos = df_clean[["titulo", "año", "duracion", "genero", "adultos"]].values.tolist()   # Convertir DataFrame directamente a lista de tuplas

query_insert = """
INSERT INTO peliculas (titulo, año, duracion, genero, adultos) 
VALUES (%s, %s, %s, %s, %s)
"""
mycursor.executemany(query_insert, datos)  # insertar muchos
cnx.commit()

print(f"{mycursor.rowcount} registros insertados")

100 registros insertados


##  Fase 4: Obtener información a partir de los datos.
Una vez que tenemos toda la información, vamos a responder las siguientes preguntas utilizando consultas en SQL